Install the following libraries

1. pip install efficientnet_pytorch

2. pip install ivtmetrics

In [1]:
!pip install efficientnet_pytorch
!pip install ivtmetrics

  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=4f48d13dc4993a635125b8e64c2b0f07c226d5c4111b5e208f20cb64bd52109b
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
  Preparing metadata (setup.py) ... - \ done
  Created wheel for ivtmetrics: filename=ivtmetrics-0.1.2-py3-none-any.whl size=15830 sha256=d963c6f00b1f6b7e06c1ab04bd11c8bc5020fa182c724f61a79ecd525e64e62c
  Stored in directory: /root/.cache/pip/wheels/3d/6e/d3/85e46407b1dbbf553b1730f4a3b10504a7fdf3e667637b965e
Successfully built ivtmetrics


In [2]:
# for dataset reading
import os
import random
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, ConcatDataset, DataLoader

# for model developing
import os
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
import torchvision.models as basemodels
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet # pip install efficientnet_pytorch
import math

# for dataset loading and training, parameters
import os
import sys
import time
import torch
import random
# import network
import argparse
import platform
import ivtmetrics
# import dataloader
import numpy as np
from torch import nn
# from torch.utils.data import DataLoader

In [3]:
# enable gpu of kaggle
import torch
print(torch.cuda.is_available())

True


# Process for the CholecT50

Original Dataloader for CholecT50, but change to T45

In [4]:
# # 加载单个视频数据及其标注(即读图片和JSON标注)
# class CholecT50Video(Dataset):
#     # 读取JSON文件
#     def __init__(self, img_dir, label_file, transform=None, target_transform=None, frame_interval=25): # 注意每隔25帧取一张图
#         with open(label_file, 'r') as f:
#             self.labels = json.load(f) # 读取JSON文件，获取标签信息
#         self.img_dir = img_dir # 图像文件所在目录
#         self.transform = transform # 图像变换函数
#         self.target_transform = target_transform # 目标变换函数（可选）
#         self.num_frames = self.labels['num_frames'] # 获取帧数
#         self.valid_indices = self._get_valid_indices() # 跳过triplet_label 为-1的图片
    
#     def _get_valid_indices(self):
#         valid_indices = []
#         for index in range(self.num_frames):
#             frame_id = str(index)
#             annotation_instances = self.labels['annotations'].get(frame_id, [])
#             triplet_labels = [inst[0] for inst in annotation_instances]
#             if all(label != -1 for label in triplet_labels):
#                 valid_indices.append(index)
#         return valid_indices
    
#     def __len__(self):
#         return len(self.valid_indices)
    
#     # 根据帧 ID 获取对应的标注。
#     def __getitem__(self, index):
# #         frame_id = str(index) # 将索引转换为字符串形式
#         valid_index = self.valid_indices[index] # 只挑选可用图片
#         frame_id = str(valid_index)
#         annotation_instances = self.labels['annotations'].get(frame_id, []) # 获取该帧对应的标签实例列表
#         basename = "{}.png".format(str(index).zfill(6)) # 构建图像文件名，格式为6位数的字符串
#         img_path = os.path.join(self.img_dir, basename) # 构建图像文件的完整路径
# #         image = Image.open(img_path) # 打开图像
#         image = Image.open(img_path).convert('RGB') # 打开图像（更新）

#         if self.transform:
#             image = self.transform(image) # 图像增强

#         # 更正：提取不同类型的标签
# #         triplet_labels = [inst[0] for inst in annotation_instances]  # 三元组标签 (0)：如果为-1，则代表图片中没有完整的工具动作和目标，此类图片需要跳过
# #         instrument_labels = [inst[1] for inst in annotation_instances]  # 工具标签 (1)
# #         verb_labels = [inst[7] for inst in annotation_instances]  # 动作标签 (7)
# # #         target_labels = [inst[8:14] for inst in annotation_instances]  # 目标标签 (8 到 12)
# #         phase_labels = [inst[14] for inst in annotation_instances]  # 阶段标签 (14)

# # Select only one annotation per frame, here we take the first one
#         if annotation_instances:
#             annotation = annotation_instances[0]
#             instrument_labels = annotation[1]
#             verb_labels = annotation[7]
#             phase_labels = annotation[14]
#         else:
#             instrument_labels = []
#             verb_labels = []
#             phase_labels = []

#         return image, (instrument_labels, verb_labels, phase_labels)


# # 数据集读取
# class CholecT50():
#     def __init__(self, 
#                 dataset_dir, # dataset_dir：数据集的根目录
#                 dataset_variant="cholect50-crossval", # dataset_variant：数据集的变体。
#                 test_fold=1, # test_fold：交叉验证时的测试集折数。 
#                 augmentation_list=['original', 'vflip', 'hflip', 'contrast', 'rot90']): # augmentation_list：数据增强列表    
#         self.dataset_dir = dataset_dir
#         self.list_dataset_variant = { # self.list_dataset_variant：保存数据集变体的描述
#             "cholect50-crossval": "for CholecT50 dataset variant with the official cross-validation splits",
#             "cholect50": "for the CholecT50 dataset with original splits used in rendezvous paper",
#         }
        
#         # 检查 dataset_variant 是否有效。
#         # 根据数据集变体选择数据分割。
#         assert dataset_variant in self.list_dataset_variant.keys(), f"{dataset_variant} is not a valid dataset variant"
#         video_split  = self.split_selector(case=dataset_variant)
#         train_videos = sum([v for k,v in video_split.items() if k!=test_fold], []) if 'crossval' in dataset_variant else video_split['train']
#         test_videos  = sum([v for k,v in video_split.items() if k==test_fold], []) if 'crossval' in dataset_variant else video_split['test']
        
#         #根据是否是交叉验证，划分训练集、验证集和测试集的视频。随后将视频编号格式化为 VIDxx。
#         if 'crossval' in dataset_variant:
#             val_videos   = train_videos[-5:]
#             train_videos = train_videos[:-5]
#         else:
#             val_videos   = video_split['val']
#         self.train_records = ['VID{}'.format(str(v).zfill(2)) for v in train_videos]
#         self.val_records   = ['VID{}'.format(str(v).zfill(2)) for v in val_videos]
#         self.test_records  = ['VID{}'.format(str(v).zfill(2)) for v in test_videos]
        
#         # 定义各种数据增强方法。根据 augmentation_list 选择要应用的增强方法。
#         self.augmentations = {
#             'original': self.no_augmentation,
#             'vflip': transforms.RandomVerticalFlip(0.4),
#             'hflip': transforms.RandomHorizontalFlip(0.4),
#             'contrast': transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0),
#             'rot90': transforms.RandomRotation(90,expand=True),
#             'brightness': transforms.RandomAdjustSharpness(sharpness_factor=1.6, p=0.5),
#             'contrast': transforms.RandomAutocontrast(p=0.5),
#         }
#         self.augmentation_list = []
#         for aug in augmentation_list:
#             self.augmentation_list.append(self.augmentations[aug])
            
#         # 获取训练和测试图像的变换方法
#         trainform, testform = self.transform()
        
#         # 调用函数构建训练、验证和测试数据集
#         self.build_train_dataset(trainform)
#         self.build_val_dataset(trainform)
#         self.build_test_dataset(testform)
    
#     # 列出数据集变体和数据增强方法。
#     def list_dataset_variants(self):
#         print(self.list_dataset_variant)
        
#     def list_augmentations(self):
#         print(self.augmentations.keys())

#     # 据不同的 case 返回相应的数据分割
#     def split_selector(self, case='cholect50'):
#         switcher = {
#             'cholect50': {
#                 'train': [1, 15, 26, 40, 52, 65, 79, 2, 18, 27, 43, 56, 66, 92, 4, 22, 31, 47, 57, 68, 96, 5, 23, 35, 48, 60, 70, 103, 13, 25, 36, 49, 62, 75, 110],
#                 'val'  : [8, 12, 29, 50, 78],
#                 'test' : [6, 51, 10, 73, 14, 74, 32, 80, 42, 111]
#             },
#             'cholect50-crossval': {
#                 1: [79,  2, 51,  6, 25, 14, 66, 23, 50, 111],
#                 2: [80, 32,  5, 15, 40, 47, 26, 48, 70,  96],
#                 3: [31, 57, 36, 18, 52, 68, 10,  8, 73, 103],
#                 4: [42, 29, 60, 27, 65, 75, 22, 49, 12, 110],
#                 5: [78, 43, 62, 35, 74,  1, 56,  4, 13,  92],
#             },
#         }
#         return switcher.get(case)

#     # 定义不进行数据增强的方法。
#     def no_augmentation(self, x):
#         return x

#     # 定义训练和测试数据的图像变换
#     def transform(self):
#         normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#         op_test   = [transforms.Resize((256, 448)), transforms.ToTensor(), normalize,]
#         op_train  = [transforms.Resize((256, 448))] + self.augmentation_list + [transforms.Resize((256, 448)), transforms.ToTensor(), normalize,]
#         testform  = transforms.Compose(op_test)
#         trainform = transforms.Compose(op_train)
#         return trainform, testform

#     # 构建训练数据集
#     def build_train_dataset(self, transform):
#         iterable_dataset = [] # 创建一个空列表，用于存储每个视频的数据集对象
#         for video in self.train_records:
#             label_file = os.path.join(self.dataset_dir, 'labels', '{}.json'.format(video)) # 构建对应视频的标签文件路径
#             if not os.path.exists(label_file):
#                 print(f"Label file not found: {label_file}")
#                 continue

#             dataset = CholecT50Video(
#                 img_dir=os.path.join(self.dataset_dir, 'videos', video),
#                 label_file=label_file,
#                 transform=transform
#             )
#             iterable_dataset.append(dataset) # 将数据集对象添加到 iterable_dataset 列表中。
#         self.train_dataset = ConcatDataset(iterable_dataset) # 使用 ConcatDataset 将所有视频的数据集对象合并成一个训练数据集

#     # 构建验证数据集
#     def build_val_dataset(self, transform):
#         iterable_dataset = []
#         for video in self.val_records:
#             label_file = os.path.join(self.dataset_dir, 'labels', '{}.json'.format(video))
#             if not os.path.exists(label_file):
#                 print(f"Label file not found: {label_file}")
#                 continue

#             dataset = CholecT50Video(
#                 img_dir=os.path.join(self.dataset_dir, 'videos', video),
#                 label_file=label_file,
#                 transform=transform
#             )
#             iterable_dataset.append(dataset) 
#         self.val_dataset = ConcatDataset(iterable_dataset) 

#     # 构建测试数据集
#     def build_test_dataset(self, transform):
#         iterable_dataset = []
#         for video in self.test_records:
#             label_file = os.path.join(self.dataset_dir, 'labels', '{}.json'.format(video))
#             if not os.path.exists(label_file):
#                 print(f"Label file not found: {label_file}")
#                 continue

#             dataset = CholecT50Video(
#                 img_dir=os.path.join(self.dataset_dir, 'videos', video),
#                 label_file=label_file,
#                 transform=transform
#             )
#             iterable_dataset.append(dataset)
#         self.test_dataset = iterable_dataset # 注意，这里 self.test_dataset 并未使用 ConcatDataset，这可能是因为测试集在某些情况下需要保持单独的视频数据集。
        
#     def build(self):
#         return (self.train_dataset, self.val_dataset, self.test_dataset)

# Process for the CholecT45

## Data loader of the CholecT45

In [5]:
# loading, augmentation, and preparation for CholecT45
class CholecT50():
    def __init__(self, 
                dataset_dir, 
                dataset_variant="cholect45-crossval",
                test_fold=1,  # choose a data fold as the test set.
                augmentation_list=['original', 'vflip', 'hflip', 'contrast', 'rot90']):
        self.dataset_dir = dataset_dir
        self.list_dataset_variant = {
            "cholect45-crossval": "for CholecT45 dataset variant with the official cross-validation splits.",
            "cholect45": "a pointer to cholect45-crossval",
        }
        assert dataset_variant in self.list_dataset_variant.keys(), print(dataset_variant, "is not a valid dataset variant")
        
        # split the dataset into training, validation, and testing
        video_split  = self.split_selector(case=dataset_variant)
        train_videos = sum([v for k,v in video_split.items() if k!=test_fold], []) if 'crossval' in dataset_variant else video_split['train']
        test_videos  = sum([v for k,v in video_split.items() if k==test_fold], []) if 'crossval' in dataset_variant else video_split['test']
        if 'crossval' in dataset_variant:
            val_videos   = train_videos[-5:]
            train_videos = train_videos[:-5]
        else:
            val_videos   = video_split['val']
        self.train_records = ['VID{}'.format(str(v).zfill(2)) for v in train_videos]
        self.val_records   = ['VID{}'.format(str(v).zfill(2)) for v in val_videos]
        self.test_records  = ['VID{}'.format(str(v).zfill(2)) for v in test_videos]
        self.augmentations = {
            'original': self.no_augumentation,
            'hflip': transforms.RandomHorizontalFlip(0.4),
            'rot90': transforms.RandomRotation(90,expand=True),
            'contrast': transforms.RandomAutocontrast(p=0.5),
            'vflip': transforms.RandomVerticalFlip(0.4),
            'contrast': transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0),
            'brightness': transforms.RandomAdjustSharpness(sharpness_factor=1.6, p=0.5),
        }
        self.augmentation_list = []
        for aug in augmentation_list:
            self.augmentation_list.append(self.augmentations[aug])
        trainform, testform = self.transform()
        self.build_train_dataset(trainform)
        self.build_val_dataset(trainform)
        self.build_test_dataset(testform)
    
    def list_dataset_variants(self):
        print(self.list_dataset_variant)

    def list_augmentations(self):
        print(self.augmentations.keys())

    def split_selector(self, case='cholect50'):
        switcher = {
            'cholect50': {
                'train': [1, 15, 26, 40, 52, 65, 79, 2, 18, 27, 43, 56, 66, 92, 4, 22, 31, 47, 57, 68, 96, 5, 23, 35, 48, 60, 70, 103, 13, 25, 36, 49, 62, 75, 110],
                'val'  : [8, 12, 29, 50, 78],
                'test' : [6, 51, 10, 73, 14, 74, 32, 80, 42, 111]
            },
            'cholect45-crossval': {
                1: [79,  2, 51,  6, 25, 14, 66, 23, 50,],
                2: [80, 32,  5, 15, 40, 47, 26, 48, 70,],
                3: [31, 57, 36, 18, 52, 68, 10,  8, 73,],
                4: [42, 29, 60, 27, 65, 75, 22, 49, 12,],
                5: [78, 43, 62, 35, 74,  1, 56,  4, 13,],
            },
        }
        return switcher.get(case)
    
    # data transformation
    def transform(self):
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        op_test   = [transforms.Resize((256, 448)), transforms.ToTensor(), normalize,]
        op_train  = [transforms.Resize((256, 448))] + self.augmentation_list + [transforms.Resize((256, 448)), transforms.ToTensor(), normalize,]
        testform  = transforms.Compose(op_test)
        trainform = transforms.Compose(op_train)
        return trainform, testform

    def build_train_dataset(self, transform):
        iterable_dataset = []
        for video in self.train_records:
            dataset = T50(img_dir = os.path.join(self.dataset_dir, 'data', video), 
                        triplet_file = os.path.join(self.dataset_dir, 'triplet', '{}.txt'.format(video)), 
                        tool_file = os.path.join(self.dataset_dir, 'instrument', '{}.txt'.format(video)),  
                        verb_file = os.path.join(self.dataset_dir, 'verb', '{}.txt'.format(video)),  
                        target_file = os.path.join(self.dataset_dir, 'target', '{}.txt'.format(video)), 
                        transform=transform)
            iterable_dataset.append(dataset)
        self.train_dataset = ConcatDataset(iterable_dataset)

    def build_val_dataset(self, transform):
        iterable_dataset = []
        for video in self.val_records:
            dataset = T50(img_dir = os.path.join(self.dataset_dir, 'data', video), 
                        triplet_file = os.path.join(self.dataset_dir, 'triplet', '{}.txt'.format(video)), 
                        tool_file = os.path.join(self.dataset_dir, 'instrument', '{}.txt'.format(video)),  
                        verb_file = os.path.join(self.dataset_dir, 'verb', '{}.txt'.format(video)),  
                        target_file = os.path.join(self.dataset_dir, 'target', '{}.txt'.format(video)), 
                        transform=transform)
            iterable_dataset.append(dataset)
        self.val_dataset = ConcatDataset(iterable_dataset)

    def build_test_dataset(self, transform):
        iterable_dataset = []
        for video in self.test_records:
            dataset = T50(img_dir = os.path.join(self.dataset_dir, 'data', video), 
                triplet_file = os.path.join(self.dataset_dir, 'triplet', '{}.txt'.format(video)), 
                tool_file = os.path.join(self.dataset_dir, 'instrument', '{}.txt'.format(video)),  
                verb_file = os.path.join(self.dataset_dir, 'verb', '{}.txt'.format(video)),  
                target_file = os.path.join(self.dataset_dir, 'target', '{}.txt'.format(video)), 
                transform=transform)
            iterable_dataset.append(dataset)
        self.test_dataset = iterable_dataset
        
    def build(self):
        return (self.train_dataset, self.val_dataset, self.test_dataset)
    
    def no_augumentation(self, x):
        return x

    
class T50(Dataset):
    def __init__(self, img_dir, triplet_file, tool_file, verb_file, target_file, transform=None, target_transform=None):
        # fix the error of "dtype=np.int" in Kaggle
#         self.triplet_labels = np.loadtxt(triplet_file, dtype=np.int, delimiter=',',)
#         self.tool_labels = np.loadtxt(tool_file, dtype=np.int, delimiter=',',)
#         self.verb_labels = np.loadtxt(verb_file, dtype=np.int, delimiter=',',)
#         self.target_labels = np.loadtxt(target_file, dtype=np.int, delimiter=',',)
        # load the label data
        self.triplet_labels = np.loadtxt(triplet_file, dtype=int, delimiter=',',)
        self.tool_labels = np.loadtxt(tool_file, dtype=int, delimiter=',',)
        self.verb_labels = np.loadtxt(verb_file, dtype=int, delimiter=',',)
        self.target_labels = np.loadtxt(target_file, dtype=int, delimiter=',',)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    
    # number of samples
    def __len__(self):
        return len(self.triplet_labels)
    
    # get an image and its label
    def __getitem__(self, index):
        triplet_label = self.triplet_labels[index, 1:]
        tool_label = self.tool_labels[index, 1:]
        verb_label = self.verb_labels[index, 1:]
        target_label = self.target_labels[index, 1:]
        basename = "{}.png".format(str(self.triplet_labels[index, 0]).zfill(6))
        img_path = os.path.join(self.img_dir, basename)
        image    = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            triplet_label = self.target_transform(triplet_label)
        return image, (tool_label, verb_label, target_label, triplet_label)

## Rendezvous with improvements

In [6]:
# Rendezvous class: the main model class, consist of an encoder and a decoder
# encoder: modules for feature extraction and processing
# decoder: processes the encoded features to generate predictions

# the dimensions of the output feature maps
OUT_HEIGHT = 8
OUT_WIDTH  = 14

class Rendezvous(nn.Module):
    # hr_output: whether to increase resolution
    # use_ln: whether to use layer normalization
    def __init__(self, basename="resnet18", num_tool=6, num_verb=10, num_target=15, num_triplet=100, layer_size=8, num_heads=4, d_model=128, hr_output=False, use_ln=False):
        super(Rendezvous, self).__init__()
        self.encoder = Encoder(basename, num_tool, num_verb, num_target, num_triplet, hr_output=hr_output)
        self.decoder = Decoder(layer_size, d_model, num_heads, num_triplet, use_ln=use_ln)    
     
    def forward(self, inputs):
        enc_i, enc_v, enc_t, enc_ivt = self.encoder(inputs)
        dec_ivt = self.decoder(enc_i, enc_v, enc_t, enc_ivt)
        return enc_i, enc_v, enc_t, dec_ivt
    

# Encoder: process input images
class Encoder(nn.Module):
    def __init__(self, basename='resnet18', num_tool=6,  num_verb=10, num_target=15, num_triplet=100, hr_output=False):
    # def after modify basemodel
#     def __init__(self, basename='efficientnet-b0', num_tool=6, num_verb=10, num_target=15, num_triplet=100, hr_output=False):
        super(Encoder, self).__init__()
        # base model of resnet18
        depth = 64 if basename == 'resnet18' else 128
        # base model of efficientnet-b0
#         depth = 128 if basename == 'efficientnet-b0' else 64
        self.basemodel  = BaseModel(basename, hr_output)
        self.wsl        = WSL(num_tool, depth)
        self.cagam      = CAGAM(num_tool, num_verb, num_target)
        self.bottleneck = Bottleneck(num_triplet) # reduce dimensionality of features
        
    def forward(self, x):
        high_x, low_x = self.basemodel(x)
        enc_i         = self.wsl(high_x)
        enc_v, enc_t  = self.cagam(high_x, enc_i[0])
        enc_ivt       = self.bottleneck(low_x)
        return enc_i, enc_v, enc_t, enc_ivt


# #%%  Decoder with the original MultiHead Attention Decoder
# class Decoder(nn.Module):
#     def __init__(self, layer_size, d_model, num_heads, num_class=100, use_ln=False):
#         super(Decoder, self).__init__()        
#         self.projection = nn.ModuleList([Projection(num_triplet=num_class, out_depth=d_model) for i in range(layer_size)])
#         self.mhma       = nn.ModuleList([MHMA(num_class=num_class, depth=d_model, num_heads=num_heads, use_ln=use_ln) for i in range(layer_size)])
#         self.ffnet      = nn.ModuleList([FFN(k=layer_size-i-1, num_class=num_class, use_ln=use_ln) for i in range(layer_size)])
#         self.classifier = Classifier(num_class)
        
#     def forward(self, enc_i, enc_v, enc_t, enc_ivt):
#         X = enc_ivt.clone()
#         for P, M, F in zip(self.projection, self.mhma, self.ffnet):
#             X = P(enc_i[0], enc_v[0], enc_t[0], X)
#             X = M(X)
#             X = F(X)
#         logits = self.classifier(X)
#         return logits


# feature extraction backbone
class BaseModel(nn.Module):   
    def __init__(self, basename='resnet18', hr_output=False, *args):
        super(BaseModel, self).__init__(*args)
        self.output_feature = {} 
#         if basename == 'resnet18':
        self.basemodel      = basemodels.resnet18(pretrained=True)     
        if hr_output: self.increase_resolution() # increase resolution of the output feature maps
        self.basemodel.layer1[1].bn2.register_forward_hook(self.get_activation('low_level_feature'))
        self.basemodel.layer4[1].bn2.register_forward_hook(self.get_activation('high_level_feature'))        
#         if basename == 'resnet50':
#             self.basemodel      = basemodels.resnet50(pretrained=True)
#             self.basemodel.layer1[2].bn2.register_forward_hook(self.get_activation('low_level_feature'))
#             self.basemodel.layer4[2].bn2.register_forward_hook(self.get_activation('high_level_feature'))
        
    # output higher resolution features
    def increase_resolution(self):  
        global OUT_HEIGHT, OUT_WIDTH  
        self.basemodel.layer3[0].conv1.stride = (1,1)
        self.basemodel.layer3[0].downsample[0].stride=(1,1)  
        self.basemodel.layer4[0].conv1.stride = (1,1)
        self.basemodel.layer4[0].downsample[0].stride=(1,1)
        OUT_HEIGHT *= 4
        OUT_WIDTH  *= 4
        print("using high resolution output ({}x{})".format(OUT_HEIGHT,OUT_WIDTH))        

    def get_activation(self, layer_name):
        def hook(module, input, output):
            self.output_feature[layer_name] = output
        return hook
    
    def forward(self, x):
        _ = self.basemodel(x)
        return self.output_feature['high_level_feature'], self.output_feature['low_level_feature']


# modified WSL
# multiple convolutional layers: extract more features, helps improve localization accuracy
# attention mechanism: enhance important features, especially for complex or multi-target
class WSL(nn.Module):
    def __init__(self, num_class, depth=64):
#     def __init__(self, num_class, depth=128): # 6.27 modify basemodel
        super(WSL, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=512, out_channels=depth, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=depth, out_channels=depth, kernel_size=3, padding=1)
        self.cam = nn.Conv2d(in_channels=depth, out_channels=num_class, kernel_size=1)
        self.elu = nn.ELU()
        self.bn1 = nn.BatchNorm2d(depth)
        self.bn2 = nn.BatchNorm2d(depth)
        self.gmp = nn.AdaptiveMaxPool2d((1, 1))
        
        # attention mechanism
        self.attention_conv = nn.Conv2d(in_channels=depth, out_channels=depth, kernel_size=1)
        self.attention_sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        feature = self.conv1(x)
        feature = self.bn1(feature)
        feature = self.elu(feature)
        
        feature = self.conv2(feature)
        feature = self.bn2(feature)
        feature = self.elu(feature)
        
        # add the attention mechanism
        attn = self.attention_conv(feature)
        attn = self.attention_sigmoid(attn)
        feature = feature * attn
        
        cam = self.cam(feature)
        logits = self.gmp(cam).squeeze(-1).squeeze(-1)
        return cam, logits

# 6.28 new: fix the error of channel number
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.out = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value):
        batch_size = query.size(0)

        # flatten the input
        query = self.query(query.view(batch_size, -1, self.embed_dim)).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        key = self.key(key.view(batch_size, -1, self.embed_dim)).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        value = self.value(value.view(batch_size, -1, self.embed_dim)).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # scaled dot-product
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attn = F.softmax(scores, dim=-1)
        context = torch.matmul(attn, value)

        # concatenate heads
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.embed_dim)
        out = self.out(context)

        return out

# add multi-head attention to the CAGAM 
class CAGAM(nn.Module):
    def __init__(self, num_tool, num_verb, num_target, in_depth=512, embed_dim=128, num_heads=4):
        super(CAGAM, self).__init__()
        self.num_tool = num_tool
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # initialize convolutional layers
        # for verb
        self.verb_context = nn.Conv2d(in_channels=in_depth, out_channels=embed_dim, kernel_size=3, padding=1)
        self.verb_query = nn.Conv2d(in_channels=embed_dim, out_channels=embed_dim, kernel_size=1)
        self.verb_tool_query = nn.Conv2d(in_channels=num_tool, out_channels=embed_dim, kernel_size=1)
        self.verb_key = nn.Conv2d(in_channels=embed_dim, out_channels=embed_dim, kernel_size=1)
        self.verb_tool_key = nn.Conv2d(in_channels=num_tool, out_channels=embed_dim, kernel_size=1)
        self.verb_cmap = nn.Conv2d(in_channels=embed_dim, out_channels=num_verb, kernel_size=1)
        
        # for target
        self.target_context = nn.Conv2d(in_channels=in_depth, out_channels=embed_dim, kernel_size=3, padding=1)
        self.target_query = nn.Conv2d(in_channels=embed_dim, out_channels=embed_dim, kernel_size=1)
        self.target_tool_query = nn.Conv2d(in_channels=num_tool, out_channels=embed_dim, kernel_size=1)
        self.target_key = nn.Conv2d(in_channels=embed_dim, out_channels=embed_dim, kernel_size=1)
        self.target_tool_key = nn.Conv2d(in_channels=num_tool, out_channels=embed_dim, kernel_size=1)
        self.target_cmap = nn.Conv2d(in_channels=embed_dim, out_channels=num_target, kernel_size=1)
        
        self.gmp = nn.AdaptiveMaxPool2d((1, 1))
        self.elu = nn.ELU()
        self.soft = nn.Softmax(dim=1)
        self.flat = nn.Flatten(2, 3)
        
        # batch normalization layers
        self.bn1 = nn.BatchNorm2d(embed_dim)
        self.bn2 = nn.BatchNorm2d(embed_dim)
        self.bn3 = nn.BatchNorm2d(embed_dim)
        self.bn4 = nn.BatchNorm2d(embed_dim)
        self.bn5 = nn.BatchNorm2d(embed_dim)
        self.bn6 = nn.BatchNorm2d(embed_dim)
        self.bn7 = nn.BatchNorm2d(embed_dim)
        self.bn8 = nn.BatchNorm2d(embed_dim)
        self.bn9 = nn.BatchNorm2d(embed_dim)
        self.bn10 = nn.BatchNorm2d(embed_dim)
        self.bn11 = nn.BatchNorm2d(embed_dim)
        self.bn12 = nn.BatchNorm2d(embed_dim)
        
        self.encoder_cagam_verb_beta = torch.nn.Parameter(torch.randn(1))
        self.encoder_cagam_target_beta = torch.nn.Parameter(torch.randn(1))
        
        self.multihead_attention = MultiHeadAttention(embed_dim=embed_dim, num_heads=num_heads)

    def get_verb(self, raw, cam):
        x = self.elu(self.bn1(self.verb_context(raw)))
        z = x.clone()
        sh = list(z.shape)
        sh[0] = -1
        q1 = self.elu(self.bn2(self.verb_query(x)))
        k1 = self.elu(self.bn3(self.verb_key(x)))
        w1 = self.flat(k1).matmul(self.flat(q1).transpose(-1, -2))
        q2 = self.elu(self.bn4(self.verb_tool_query(cam)))
        k2 = self.elu(self.bn5(self.verb_tool_key(cam)))
        w2 = self.flat(k2).matmul(self.flat(q2).transpose(-1, -2))
        attention = (w1 * w2) / torch.sqrt(torch.tensor(sh[-1], dtype=torch.float32))
        attention = self.soft(attention)
        v = self.flat(z)
        e = (attention.matmul(v) * self.encoder_cagam_verb_beta).reshape(sh)
        e = self.bn6(e + z)
        e = self.multihead_attention(e.view(-1, self.embed_dim), e.view(-1, self.embed_dim), e.view(-1, self.embed_dim))  # Apply multi-head attention
        cmap = self.verb_cmap(e.view(sh))
        y = self.gmp(cmap).squeeze(-1).squeeze(-1)
        return cmap, y

    def get_target(self, raw, cam):
        x = self.elu(self.bn7(self.target_context(raw)))
        z = x.clone()
        sh = list(z.shape)
        sh[0] = -1
        q1 = self.elu(self.bn8(self.target_query(x)))
        k1 = self.elu(self.bn9(self.target_key(x)))
        w1 = self.flat(k1).transpose(-1, -2).matmul(self.flat(q1))
        q2 = self.elu(self.bn10(self.target_tool_query(cam)))
        k2 = self.elu(self.bn11(self.target_tool_key(cam)))
        w2 = self.flat(k2).transpose(-1, -2).matmul(self.flat(q2))
        attention = (w1 * w2) / torch.sqrt(torch.tensor(sh[-1], dtype=torch.float32))
        attention = self.soft(attention)
        v = self.flat(z)
        e = (v.matmul(attention) * self.encoder_cagam_target_beta).reshape(sh)
        e = self.bn12(e + z)
        e = self.multihead_attention(e.view(-1, self.embed_dim), e.view(-1, self.embed_dim), e.view(-1, self.embed_dim))  # Apply multi-head attention
        cmap = self.target_cmap(e.view(sh))
        y = self.gmp(cmap).squeeze(-1).squeeze(-1)
        return cmap, y

    def forward(self, x, cam):
        cam_v, logit_v = self.get_verb(x, cam)
        cam_t, logit_t = self.get_target(x, cam)
        return (cam_v, logit_v), (cam_t, logit_t)

# reduce dimensionality of features
class Bottleneck(nn.Module):
    def __init__(self, num_class):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=64, out_channels=256, stride=(2,2), kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=num_class, kernel_size=1)
        self.elu   = nn.ELU()
        self.bn1   = nn.BatchNorm2d(256)
        self.bn2   = nn.BatchNorm2d(num_class)
        
    def forward(self, x):
        feature = self.conv1(x)
        feature = self.bn1(feature)
        feature = self.elu(feature)
        feature = self.conv2(feature)
        feature = self.bn2(feature)
        feature = self.elu(feature)
        return feature

# projection function
class Projection(nn.Module):
    def __init__(self, num_tool=6, num_verb=10, num_target=15, num_triplet=100, out_depth=128):
        super(Projection, self).__init__()
        self.ivt_value = nn.Conv2d(in_channels=num_triplet, out_channels=out_depth, kernel_size=1)
        self.i_value   = nn.Conv2d(in_channels=num_tool, out_channels=out_depth, kernel_size=1)        
        self.v_value   = nn.Conv2d(in_channels=num_verb, out_channels=out_depth, kernel_size=1)
        self.t_value   = nn.Conv2d(in_channels=num_target, out_channels=out_depth, kernel_size=1)       
        self.ivt_query = nn.Linear(in_features=num_triplet, out_features=out_depth)
        self.dropout   = nn.Dropout(p=0.3)
        self.ivt_key   = nn.Linear(in_features=num_triplet, out_features=out_depth)
        self.i_key     = nn.Linear(in_features=num_tool, out_features=out_depth)
        self.v_key     = nn.Linear(in_features=num_verb, out_features=out_depth)
        self.t_key     = nn.Linear(in_features=num_target, out_features=out_depth)
        self.gap       = nn.AdaptiveAvgPool2d((1,1))
        self.elu       = nn.ELU()          
        self.bn1       = nn.BatchNorm1d(out_depth)
        self.bn2       = nn.BatchNorm1d(out_depth)
        self.bn3       = nn.BatchNorm2d(out_depth)
        self.bn4       = nn.BatchNorm1d(out_depth)
        self.bn5       = nn.BatchNorm2d(out_depth)
        self.bn6       = nn.BatchNorm1d(out_depth)
        self.bn7       = nn.BatchNorm2d(out_depth)
        self.bn8       = nn.BatchNorm1d(out_depth)
        self.bn9       = nn.BatchNorm2d(out_depth) 
        
    def forward(self, cam_i, cam_v, cam_t, X):  
        q = self.elu(self.bn1(self.ivt_query(self.dropout(self.gap(X).squeeze(-1).squeeze(-1)))))  
        k = self.elu(self.bn2(self.ivt_key(self.gap(X).squeeze(-1).squeeze(-1))) )
        v = self.bn3(self.ivt_value(X)) 
        k1 = self.elu(self.bn4(self.i_key(self.gap(cam_i).squeeze(-1).squeeze(-1))) )
        v1 = self.elu(self.bn5(self.i_value(cam_i)) )
        k2 = self.elu(self.bn6(self.v_key(self.gap(cam_v).squeeze(-1).squeeze(-1))))
        v2 = self.elu(self.bn7(self.v_value(cam_v)) )
        k3 = self.elu(self.bn8(self.t_key(self.gap(cam_t).squeeze(-1).squeeze(-1))))
        v3 = self.elu(self.bn9(self.t_value(cam_t)))
        sh = list(v1.shape)
        v  = self.elu(F.interpolate(v, (sh[2],sh[3])))
        X  = self.elu(F.interpolate(X, (sh[2],sh[3])))
        return (X, (k1,v1), (k2,v2), (k3,v3), (q,k,v))

# Original multi-head of self and cross attention
# class MHMA(nn.Module):
#     def __init__(self, depth, num_class=100, num_heads=4, use_ln=False):
#         super(MHMA, self).__init__()        
#         self.concat = nn.Conv2d(in_channels=depth*num_heads, out_channels=num_class, kernel_size=3, padding=1)
#         self.bn     = nn.BatchNorm2d(num_class)
#         self.ln     = nn.LayerNorm([num_class, OUT_HEIGHT, OUT_WIDTH]) if use_ln else nn.BatchNorm2d(num_class)
#         self.elu    = nn.ELU()    
#         self.soft   = nn.Softmax(dim=1) 
#         self.heads  = num_heads
        
#     def scale_dot_product(self, key, value, query):
#         dk        = torch.sqrt(torch.tensor(list(key.shape)[-2], dtype=torch.float32))
#         affinity  = key.matmul(query.transpose(-1,-2))                        
#         attn_w    = affinity / dk              
#         attn_w    = self.soft(attn_w)
#         attention = attn_w.matmul(value) 
#         return attention
    
#     def forward(self, inputs):
#         (X, (k1,v1), (k2,v2), (k3,v3), (q,k,v)) = inputs     
#         query = torch.stack([q]*self.heads, dim=1) # [B,Head,D]
#         query = query.unsqueeze(dim=-1) # [B,Head,D,1]        
#         key   = torch.stack([k,k1,k2,k3], dim=1) # [B,Head,D]
#         key   = key.unsqueeze(dim=-1) # [B,Head,D,1]        
#         value = torch.stack([v,v1,v2,v3], dim=1) # [B,Head,D,H,W]
#         dims  = list(value.shape) # [B,Head,D,H,W]
#         value = value.reshape([-1,dims[1],dims[2],dims[3]*dims[4]])# [B,Head,D,HW]          
#         attn  = self.scale_dot_product(key, value, query)  # [B,Head,D,HW]
#         attn  = attn.reshape([-1,dims[1]*dims[2],dims[3],dims[4]]) # [B,DHead,H,W]
#         mha   = self.elu(self.bn(self.concat(attn)))
#         mha   = self.ln(mha + X.clone())  
#         return mha

# MHMA with LSTM
class MHMA(nn.Module):
    def __init__(self, depth, num_class=100, num_heads=4, use_ln=True, lstm_hidden_size=128, lstm_layers=1):
        super(MHMA, self).__init__()
        self.concat = nn.Conv2d(in_channels=depth*num_heads, out_channels=num_class, kernel_size=3, padding=1)
        self.bn     = nn.BatchNorm2d(num_class)
        self.ln     = nn.LayerNorm([num_class, OUT_HEIGHT, OUT_WIDTH]) if use_ln else nn.BatchNorm2d(num_class)
        self.elu    = nn.ELU()
        self.soft   = nn.Softmax(dim=1)
        self.heads  = num_heads
        self.dropout = nn.Dropout(0.1)  # add Dropout

        # define LSTM
        self.lstm = nn.LSTM(input_size=num_class * OUT_HEIGHT * OUT_WIDTH,
                            hidden_size=lstm_hidden_size,
                            num_layers=lstm_layers,
                            batch_first=True)
        
        # LSTM Hidden Size for convolutional output
        self.fc = nn.Linear(lstm_hidden_size, num_class * OUT_HEIGHT * OUT_WIDTH)

    def scale_dot_product(self, key, value, query):
        dk        = torch.sqrt(torch.tensor(list(key.shape)[-2], dtype=torch.float32))
        affinity  = key.matmul(query.transpose(-1,-2))
        attn_w    = affinity / dk
        attn_w    = self.soft(attn_w)
        attention = attn_w.matmul(value)
        attention = self.dropout(attention)  # add Dropout
        return attention

    def forward(self, inputs):
        (X, (k1,v1), (k2,v2), (k3,v3), (q,k,v)) = inputs
        query = torch.stack([q]*self.heads, dim=1)
        query = query.unsqueeze(dim=-1)
        key   = torch.stack([k,k1,k2,k3], dim=1) 
        key   = key.unsqueeze(dim=-1)
        value = torch.stack([v,v1,v2,v3], dim=1)
        dims  = list(value.shape)
        value = value.reshape([-1,dims[1],dims[2],dims[3]*dims[4]])
        attn  = self.scale_dot_product(key, value, query)
        attn  = attn.reshape([-1,dims[1]*dims[2],dims[3],dims[4]])
        mha   = self.elu(self.bn(self.concat(attn)))
        mha   = self.ln(mha + X.clone())

        # flat the output of the multi-head attention as the input of the LSTM
        batch_size = mha.size(0)
        mha_flatten = mha.view(batch_size, -1)  # [B, num_class * H * W]
        mha_flatten = mha_flatten.unsqueeze(1)  # [B, 1, num_class * H * W]

        # go through LSTM
        lstm_out, _ = self.lstm(mha_flatten)  # [B, 1, lstm_hidden_size]
        lstm_out = lstm_out.squeeze(1)  # [B, lstm_hidden_size]

        # fcl to resize the output
        lstm_out = self.fc(lstm_out)  # [B, num_class * H * W]
        lstm_out = lstm_out.view(batch_size, -1, OUT_HEIGHT, OUT_WIDTH)  # [B, num_class, H, W]
        
        return lstm_out
    
# feed-forward layer
class FFN(nn.Module):
    def __init__(self, k, num_class=100, use_ln=False):
        super(FFN, self).__init__()
        def Ignore(x): return x
        self.conv1 = nn.Conv2d(in_channels=num_class, out_channels=num_class, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=num_class, out_channels=num_class, kernel_size=1) 
        self.elu1  = nn.ELU() 
        self.elu2  = nn.ELU() if k>0 else Ignore     
        self.bn1   = nn.BatchNorm2d(num_class)    
        self.bn2   = nn.BatchNorm2d(num_class)
        self.ln    = nn.LayerNorm([num_class, OUT_HEIGHT, OUT_WIDTH]) if use_ln else nn.BatchNorm2d(num_class)
        
    def forward(self, inputs,):
        x  = self.elu1(self.bn1(self.conv1(inputs)))
        x  = self.elu2(self.bn2(self.conv2(x)))
        x  = self.ln(x + inputs.clone())
        return x

# classification layer
class Classifier(nn.Module):
    def __init__(self, layer_size, num_class=100):
        super(Classifier, self).__init__()
        self.gmp = nn.AdaptiveMaxPool2d((1,1)) 
        self.mlp = nn.Linear(in_features=num_class, out_features=num_class)     
        
    def forward(self, inputs):
        x = self.gmp(inputs).squeeze(-1).squeeze(-1)
        y = self.mlp(x)
        return y
    
# DC
# add dynamic convolution module
# combines multiple kernels in an adaptive manner using learned weights
class DynamicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, num_kernels=4):
        super(DynamicConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.num_kernels = num_kernels

        self.weight = nn.Parameter(torch.randn(num_kernels, out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.randn(num_kernels, out_channels))

        self.kernel_weights = nn.Parameter(torch.randn(num_kernels))

    def forward(self, x):
        conv_outs = []
        for i in range(self.num_kernels):
            conv_out = F.conv2d(x, self.weight[i], self.bias[i], stride=self.stride, padding=self.padding)
            conv_outs.append(conv_out.unsqueeze(0))

        conv_outs = torch.cat(conv_outs, dim=0)

        kernel_weights = F.softmax(self.kernel_weights, dim=0)

        out = 0
        for i in range(self.num_kernels):
            out += kernel_weights[i] * conv_outs[i]

        return out

# extend the dynamic convolution by adding batch normalization and ELU
class DynamicConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, num_kernels=4):
        super(DynamicConvolution, self).__init__()
        self.dynamic_conv = DynamicConv2d(in_channels, out_channels, kernel_size, stride, padding, num_kernels=num_kernels)
        self.elu = nn.ELU()
        self.bn = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = self.dynamic_conv(x)
        x = self.bn(x)
        x = self.elu(x)
        return x

# add DC to the decoder
class Decoder(nn.Module):
    def __init__(self, layer_size, d_model, num_heads, num_class=100, use_ln=False):
        super(Decoder, self).__init__()        
        self.projection = nn.ModuleList([Projection(num_triplet=num_class, out_depth=d_model) for i in range(layer_size)])
        self.mhma = nn.ModuleList([MHMA(num_class=num_class, depth=d_model, num_heads=num_heads, use_ln=use_ln) for i in range(layer_size)])
        self.dynamic_conv = nn.ModuleList([DynamicConvolution(num_class, num_class) for i in range(layer_size)])
        self.ffnet = nn.ModuleList([FFN(k=layer_size-i-1, num_class=num_class, use_ln=use_ln) for i in range(layer_size)])
        self.classifier = Classifier(num_class)
        
    def forward(self, enc_i, enc_v, enc_t, enc_ivt):
        X = enc_ivt.clone()
        for P, M, D, F in zip(self.projection, self.mhma, self.dynamic_conv, self.ffnet):
            X = P(enc_i[0], enc_v[0], enc_t[0], X)
            X = M(X)
            X = D(X)
            X = F(X)
        logits = self.classifier(X)
        return logits

# Set up some parameters

In [7]:
parser = argparse.ArgumentParser() # set up a command-line argument parser, also for Kaggle
# for the model
parser.add_argument('--model', type=str, default='rendezvous', choices=['rendezvous'], help='Model name?')
parser.add_argument('--version', type=int, default=0,  help='Model version control (for keeping several versions)') 
parser.add_argument('--hr_output', action='store_true', help='Whether to use higher resolution output (32x56) or not (8x14). Default: False')
parser.add_argument('--use_ln', action='store_true', help='Whether to use layer norm or batch norm in AddNorm() function. Default: False')
parser.add_argument('--decoder_layer', type=int, default=8, help='Number of MHMA layers ') 
# for tasks
parser.add_argument('-t', '--train', action='store_true', help='to train.')
parser.add_argument('-e', '--test',  action='store_true', help='to test')
parser.add_argument('--val_interval', type=int, default=1,  help='(for hp tuning). Epoch interval to evaluate on validation data. set -1 for only after final epoch, or a number higher than the total epochs to not validate.')
# for data
parser.add_argument('--data_dir', type=str, default='/kaggle/input/cholect45/CholecT45', help='path to dataset?')
parser.add_argument('--dataset_variant', type=str, default='cholect45-crossval', choices=['cholect50', 'cholect45', 'cholect50-challenge', 'cholect50-crossval', 'cholect45-crossval'], help='Variant of the dataset to use')
parser.add_argument('-k', '--kfold', type=int, default=1,  choices=[1,2,3,4,5,], help='The test split in k-fold cross-validation')
parser.add_argument('--image_width', type=int, default=448, help='Image width ')  
parser.add_argument('--image_height', type=int, default=256, help='Image height ')  
parser.add_argument('--image_channel', type=int, default=3, help='Image channels ')  
parser.add_argument('--num_tool_classes', type=int, default=6, help='Number of tool categories')
parser.add_argument('--num_verb_classes', type=int, default=10, help='Number of verb categories')
parser.add_argument('--num_target_classes', type=int, default=15, help='Number of target categories')
parser.add_argument('--num_triplet_classes', type=int, default=100, help='Number of triplet categories')
parser.add_argument('--augmentation_list', type=str, nargs='*', default=['original', 'vflip', 'hflip', 'contrast', 'rot90'], help='List augumentation styles (see dataloader.py for list of supported styles).')
# hp
parser.add_argument('-b', '--batch', type=int, default=32,  help='The size of sample training batch')
# parser.add_argument('--epochs', type=int, default=100,  help='How many training epochs?')
parser.add_argument('--epochs', type=int, default=1,  help='How many training epochs?') # test for 1 epoch
parser.add_argument('-w', '--warmups', type=int, nargs='+', default=[9,18,58], help='List warmup epochs for tool, verb-target, triplet respectively')
parser.add_argument('-l', '--initial_learning_rates', type=float, nargs='+', default=[0.01, 0.01, 0.01], help='List learning rates for tool, verb-target, triplet respectively')
parser.add_argument('--weight_decay', type=float, default=1e-5,  help='L2 regularization weight decay constant')
parser.add_argument('--decay_steps', type=int, default=10,  help='Step to exponentially decay')
parser.add_argument('--decay_rate', type=float, default=0.99,  help='Learning rates weight decay rate')
parser.add_argument('--momentum', type=float, default=0.95,  help="Optimizer's momentum")
parser.add_argument('--power', type=float, default=0.1,  help='Learning rates weight decay power')
# weights
parser.add_argument('--pretrain_dir', type=str, default='', help='path to pretrain_weight?')
parser.add_argument('--test_ckpt', type=str, default=None, help='path to model weight for testing')
# device (kaggle only has one gpu)
parser.add_argument('--gpu', type=str, default="0",  help='The gpu device to use. To use multiple gpu put all the device ids comma-separated, e.g: "0,1,2" ')
FLAGS, unparsed = parser.parse_known_args()

# set up global variables
is_train        = FLAGS.train
is_test         = FLAGS.test
dataset_variant = FLAGS.dataset_variant
data_dir        = FLAGS.data_dir
kfold           = FLAGS.kfold if "crossval" in dataset_variant else 0
version         = FLAGS.version
hr_output       = FLAGS.hr_output
use_ln          = FLAGS.use_ln
batch_size      = FLAGS.batch
pretrain_dir    = FLAGS.pretrain_dir
test_ckpt       = FLAGS.test_ckpt
weight_decay    = FLAGS.weight_decay
learning_rates  = FLAGS.initial_learning_rates
warmups         = FLAGS.warmups
decay_steps     = FLAGS.decay_steps
decay_rate      = FLAGS.decay_rate
power           = FLAGS.power
momentum        = FLAGS.momentum
epochs          = FLAGS.epochs
gpu             = FLAGS.gpu
image_height    = FLAGS.image_height
image_width     = FLAGS.image_width
image_channel   = FLAGS.image_channel
num_triplet     = FLAGS.num_triplet_classes
num_tool        = FLAGS.num_tool_classes
num_verb        = FLAGS.num_verb_classes
num_target      = FLAGS.num_target_classes
val_interval    = FLAGS.epochs-1 if FLAGS.val_interval==-1 else FLAGS.val_interval
set_chlg_eval   = True if "challenge" in dataset_variant else False # To observe challenge evaluation protocol
gpu             = ",".join(str(FLAGS.gpu).split(","))
decodelayer     = FLAGS.decoder_layer
addnorm         = "layer" if use_ln else "batch"
modelsize       = "high" if hr_output else "low"
FLAGS.multigpu  = len(gpu) > 1  # not yet implemented!
mheaders        = ["","l", "cholect", "k"]
margs           = [FLAGS.model, decodelayer, dataset_variant, kfold]
wheaders        = ["norm", "res"]
wargs           = [addnorm, modelsize]
modelname       = "_".join(["{}{}".format(x,y) for x,y in zip(mheaders, margs) if len(str(y))])+"_"+\
                  "_".join(["{}{}".format(x,y) for x,y in zip(wargs, wheaders) if len(str(x))])
model_dir       = "./__checkpoint__/run_{}".format(version)
if not os.path.exists(model_dir): os.makedirs(model_dir)
resume_ckpt     = None
# ckpt_path       = os.path.join(model_dir, '{}.pth'.format(modelname))
ckpt_path = os.path.join('/kaggle/working', '{}.pth'.format(modelname))
logfile         = os.path.join(model_dir, '{}.log'.format(modelname))
data_augmentations      = FLAGS.augmentation_list 
iterable_augmentations  = []
print("Configuring network ...")

# helpers
def assign_gpu(gpu=None):  
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu) 
    os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1' 
    

def get_weight_balancing(case='cholect50'):
    # 50:   cholecT50, data splits as used in rendezvous paper
    # 50ch: cholecT50, data splits as used in CholecTriplet challenge
    # 45cv: cholecT45, official data splits (cross-val)
    # 50cv: cholecT50, official data splits (cross-val)
    switcher = {
        'cholect50': {
            'tool'  :   [0.08084519, 0.81435289, 0.10459284, 2.55976864, 1.630372490, 1.29528455],
            'verb'  :   [0.31956735, 0.07252306, 0.08111481, 0.81137309, 1.302895320, 2.12264151, 1.54109589, 8.86363636, 12.13692946, 0.40462028],
            'target':   [0.06246232, 1.00000000, 0.34266478, 0.84750219, 14.80102041, 8.73795181, 1.52845100, 5.74455446, 0.285756500, 12.72368421, 0.6250808,  3.85771277, 6.95683453, 0.84923888, 0.40130032]
        },
        'cholect50-challenge': {
            'tool':     [0.08495163, 0.88782288, 0.11259564, 2.61948830, 1.784866470, 1.144624170],
            'verb':     [0.39862805, 0.06981640, 0.08332925, 0.81876204, 1.415868390, 2.269359150, 1.28428410, 7.35822511, 18.67857143, 0.45704490],
            'target':   [0.07333818, 0.87139287, 0.42853950, 1.00000000, 17.67281106, 13.94545455, 1.44880997, 6.04889590, 0.326188650, 16.82017544, 0.63577586, 6.79964539, 6.19547658, 0.96284208, 0.51559559]
        },
        'cholect45-crossval': {
            1: {
                'tool':     [0.08165644, 0.91226868, 0.10674758, 2.85418156, 1.60554885, 1.10640067],
                'verb':     [0.37870137, 0.06836869, 0.07931255, 0.84780024, 1.21880342, 2.52836879, 1.30765704, 6.88888889, 17.07784431, 0.45241117],
                'target':   [0.07149629, 1.0, 0.41013597, 0.90458015, 13.06299213, 12.06545455, 1.5213205, 5.04255319, 0.35808332, 45.45205479, 0.67493897, 7.04458599, 9.14049587, 0.97330595, 0.52633249]
                },
            2: {
                'tool':     [0.0854156, 0.89535362, 0.10995253, 2.74936869, 1.78264429, 1.13234529],
                'verb':     [0.36346863, 0.06771776, 0.07893261, 0.82842725, 1.33892161, 2.13049748, 1.26120359, 5.72674419, 19.7, 0.43189126],
                'target':   [0.07530655, 0.97961957, 0.4325135, 0.99393438, 15.5387931, 14.5951417, 1.53862569, 6.01836394, 0.35184462, 15.81140351, 0.709506, 5.79581994, 8.08295964, 1.0, 0.52689272]
            },
            3: {
                "tool" :   [0.0915228, 0.89714969, 0.12057004, 2.72128174, 1.94092281, 1.12948557],
                "verb" :   [0.43636862, 0.07558554, 0.0891017, 0.81820519, 1.53645582, 2.31924198, 1.28565657, 6.49387755, 18.28735632, 0.48676763],
                "target" : [0.06841828, 0.90980736, 0.38826607, 1.0, 14.3640553, 12.9875, 1.25939394, 5.38341969, 0.29060227, 13.67105263, 0.59168565, 6.58985201, 5.72977941, 0.86824513, 0.47682423]

            },
            4: {
                'tool':     [0.08222218, 0.85414117, 0.10948695, 2.50868784, 1.63235867, 1.20593318],
                'verb':     [0.41154261, 0.0692142, 0.08427214, 0.79895288, 1.33625219, 2.2624166, 1.35343681, 7.63, 17.84795322, 0.43970609],
                'target':   [0.07536126, 0.85398445, 0.4085784, 0.95464422, 15.90497738, 18.5978836, 1.55875831, 5.52672956, 0.33700863, 15.41666667, 0.74755423, 5.4921875, 6.11304348, 1.0, 0.50641118],
            },
            5: {
                'tool':     [0.0804654, 0.92271157, 0.10489631, 2.52302243, 1.60074906, 1.09141982],
                'verb':     [0.50710436, 0.06590258, 0.07981184, 0.81538866, 1.29267277, 2.20525568, 1.29699248, 7.32311321, 25.45081967, 0.46733895],
                'target':   [0.07119395, 0.87450495, 0.43043372, 0.86465981, 14.01984127, 23.7114094, 1.47577277, 5.81085526, 0.32129865, 22.79354839, 0.63304067, 6.92745098, 5.88833333, 1.0, 0.53175798]
            }
        },
        'cholect50-crossval': {
            1:{
                'tool':     [0.0828851, 0.8876, 0.10830995, 2.93907285, 1.63884786, 1.14499484],
                'verb':     [0.29628942, 0.07366916, 0.08267971, 0.83155428, 1.25402434, 2.38358209, 1.34938741, 7.56872038, 12.98373984, 0.41502079],
                'target':   [0.06551745, 1.0, 0.36345711, 0.82434783, 13.06299213, 8.61818182, 1.4017744, 4.62116992, 0.32822238, 45.45205479, 0.67343211, 4.13200498, 8.23325062, 0.88527215, 0.43113306],

            },
            2:{
                'tool':     [0.08586283, 0.87716737, 0.11068887, 2.84210526, 1.81016949, 1.16283571],
                'verb':     [0.30072757, 0.07275414, 0.08350168, 0.80694143, 1.39209979, 2.22754491, 1.31448763, 6.38931298, 13.89211618, 0.39397505],
                'target':   [0.07056703, 1.0, 0.39451115, 0.91977006, 15.86206897, 9.68421053, 1.44483706, 5.44378698, 0.31858714, 16.14035088, 0.7238395, 4.20571429, 7.98264642, 0.91360477, 0.43304307],
            },
            3:{
            'tool':      [0.09225068, 0.87856006, 0.12195811, 2.82669323, 1.97710987, 1.1603972],
                'verb':     [0.34285159, 0.08049804, 0.0928239, 0.80685714, 1.56125608, 2.23984772, 1.31471136, 7.08835341, 12.17241379, 0.43180428],
                'target':   [0.06919395, 1.0, 0.37532866, 0.9830703, 15.78801843, 8.99212598, 1.27597765, 5.36990596, 0.29177312, 15.02631579, 0.64935557, 5.08308605, 5.86643836, 0.86580743, 0.41908257], 
            },
            4:{
                'tool':     [0.08247885, 0.83095539, 0.11050268, 2.58193042, 1.64497676, 1.25538881],
                'verb':     [0.31890981, 0.07380354, 0.08804592, 0.79094077, 1.35928144, 2.17017208, 1.42947103, 8.34558824, 13.19767442, 0.40666428],
                'target':   [0.07777646, 0.95894072, 0.41993829, 0.95592153, 17.85972851, 12.49050633, 1.65701092, 5.74526929, 0.33763901, 17.31140351, 0.83747083, 3.95490982, 6.57833333, 1.0, 0.47139615],
            },
            5:{
                'tool':     [0.07891691, 0.89878025, 0.10267677, 2.53805556, 1.60636428, 1.12691169],
                'verb':     [0.36420961, 0.06825313, 0.08060635, 0.80956984, 1.30757221, 2.09375, 1.33625848, 7.9009434, 14.1350211, 0.41429631],
                'target':   [0.07300329, 0.97128713, 0.42084942, 0.8829883, 15.57142857, 19.42574257, 1.56521739, 5.86547085, 0.32732733, 25.31612903, 0.70171674, 4.55220418, 6.13125, 1.0, 0.48528321],
            }
        }
    }
    return switcher.get(case)

Configuring network ...


In [8]:
def train_loop(dataloader, model, activation, loss_fn_i, loss_fn_v, loss_fn_t, loss_fn_ivt, optimizers, scheduler, epoch):
    start = time.time() 
    total_loss_i = 0.0
    total_loss_v = 0.0
    total_loss_t = 0.0
    total_loss_ivt = 0.0
    total_samples = 0

    for batch, (img, (y1, y2, y3, y4)) in enumerate(dataloader):
        img, y1, y2, y3, y4 = img.cuda(), y1.cuda(), y2.cuda(), y3.cuda(), y4.cuda()        
        model.train()        
        tool, verb, target, triplet = model(img)
        cam_i, logit_i  = tool
        cam_v, logit_v  = verb
        cam_t, logit_t  = target
        logit_ivt       = triplet                
        loss_i          = loss_fn_i(logit_i, y1.float())
        loss_v          = loss_fn_v(logit_v, y2.float())
        loss_t          = loss_fn_t(logit_t, y3.float())
        loss_ivt        = loss_fn_ivt(logit_ivt, y4.float())  
        loss            = (loss_i) + (loss_v) + (loss_t) + loss_ivt 
        
        # total loss
        batch_size = img.size(0)
        total_loss_i += loss_i.item() * batch_size
        total_loss_v += loss_v.item() * batch_size
        total_loss_t += loss_t.item() * batch_size
        total_loss_ivt += loss_ivt.item() * batch_size
        total_samples += batch_size

        # back-propagation 
        for param in model.parameters():
            param.grad = None
        loss.backward()
        for opt in optimizers:
            opt.step()

        print(f'Batch {batch+1}/{len(dataloader)} | Losses => i: [{loss_i.item():.4f}] v: [{loss_v.item():.4f}] t: [{loss_t.item():.4f}] ivt: [{loss_ivt.item():.4f}]')

    # update learning rate
    for sch in scheduler:
        sch.step()
    
    # average losses for each epoch
    avg_loss_i = total_loss_i / total_samples
    avg_loss_v = total_loss_v / total_samples
    avg_loss_t = total_loss_t / total_samples
    avg_loss_ivt = total_loss_ivt / total_samples

    print(f'Epoch {epoch+1} completed | Average Losses--- i: [{avg_loss_i:.4f}] v: [{avg_loss_v:.4f}] t: [{avg_loss_t:.4f}] ivt: [{avg_loss_ivt:.4f}] | eta: {(time.time() - start):.2f} secs')

    # average precision for each epoch
    with torch.no_grad():
        mAP.reset_global()
        for img, (y1, y2, y3, y4) in dataloader:
            img, y1, y2, y3, y4 = img.cuda(), y1.cuda(), y2.cuda(), y3.cuda(), y4.cuda()
            tool, verb, target, triplet = model(img)
            logit_ivt = triplet
            mAP.update(y4.float().detach().cpu(), activation(logit_ivt).detach().cpu())
        avg_precision = mAP.compute_video_AP()['mAP']
        print(f'Average Precision for Epoch {epoch+1}: {avg_precision:.4f}')


def calculate_top_n_accuracy(logits, targets, n):
    # print log information
#     print(f"Logits shape: {logits.shape}")
#     print(f"Targets shape: {targets.shape}")

    _, top_n_pred = logits.topk(n, dim=1)
#     print(f"Top-{n} predictions shape: {top_n_pred.shape}")

    # convert targets from one-hot encoding to class indices. 
    targets = torch.argmax(targets, dim=1)
#     print(f"Converted targets shape: {targets.shape}")

    if top_n_pred.size(0) != targets.size(0):
        raise ValueError("The batch size does not match between the Logits and the target")

    # calculate top-n accuracy
    correct_top_n = top_n_pred.eq(targets.view(-1, 1).expand_as(top_n_pred))
    top_n_accuracy = correct_top_n.any(dim=1).float().mean().item()
    return top_n_accuracy

def test_loop(dataloader, model, activation, final_eval=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)   
    mAP.reset()  
    
    top5_accuracy = 0.0
    top10_accuracy = 0.0
    total_samples = 0
    
    if final_eval and not set_chlg_eval:
        mAPv.reset() 
        mAPt.reset() 
        mAPi.reset() 
    with torch.no_grad():
        for batch, (img, (y1, y2, y3, y4)) in enumerate(dataloader):
            img, y1, y2, y3, y4 = img.cuda(), y1.cuda(), y2.cuda(), y3.cuda(), y4.cuda()            
            model.eval()  
            tool, verb, target, triplet = model(img)
            
            # calculate Top-5 and Top-10 accuracies for triplet predictions
            triplet_logits = activation(triplet)
            top5_accuracy += calculate_top_n_accuracy(triplet_logits, y4, 5) * img.size(0)
            top10_accuracy += calculate_top_n_accuracy(triplet_logits, y4, 10) * img.size(0)
            total_samples += img.size(0)
            
            if final_eval and not set_chlg_eval:
                cam_i, logit_i = tool
                cam_v, logit_v = verb
                cam_t, logit_t = target
                mAPi.update(y1.float().detach().cpu(), activation(logit_i).detach().cpu())
                mAPv.update(y2.float().detach().cpu(), activation(logit_v).detach().cpu())
                mAPt.update(y3.float().detach().cpu(), activation(logit_t).detach().cpu())
            mAP.update(y4.float().detach().cpu(), activation(triplet).detach().cpu())
    mAP.video_end() 
    if final_eval and not set_chlg_eval:
        mAPv.video_end()
        mAPt.video_end()
        mAPi.video_end()
        
    # Top-5 and Top-10 accuracies
    top5_accuracy /= total_samples
    top10_accuracy /= total_samples
#     print(f"Top-5 Accuracy: {top5_accuracy:.4f}")
#     print(f"Top-10 Accuracy: {top10_accuracy:.4f}")
    return top5_accuracy, top10_accuracy, total_samples


def weight_mgt(score, epoch):
    # hyperparameter selection based on validation set
    global benchmark
    if score > benchmark.item():
        torch.save(model.state_dict(), ckpt_path)
        benchmark = score
#         print(f'>>> Saving checkpoint for epoch {epoch+1} at {ckpt_path}, time {time.ctime()} ', file=open(logfile, 'a+'))  
        return "increased"
    else:
        return "decreased"

# during testing or resuming
if os.path.exists(ckpt_path):
    model.load_state_dict(torch.load(ckpt_path))
    print(f'Model loaded from checkpoint at {ckpt_path}')

##%% assign device and set debugger options
assign_gpu(gpu=gpu)
np.seterr(divide='ignore', invalid='ignore') # ignore divide-by-zero and invalid operations.
torch.autograd.set_detect_anomaly(False) # disable anomaly detection

# disable profiling and NVTX emission for debugging
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
# torch.set_deterministic(True)

## Load the dataset

In [9]:
# variant and split selection (original paper used different augumentation per epoch)
# dataset = dataloader.CholecT50( 
dataset = CholecT50( 
            dataset_dir=data_dir, 
            dataset_variant=dataset_variant,
            test_fold=kfold,
            augmentation_list=data_augmentations,
            )

# build dataset
train_dataset, val_dataset, test_dataset = dataset.build()

# train and val data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, prefetch_factor=3*batch_size, num_workers=3, pin_memory=True, persistent_workers=True, drop_last=False)
val_dataloader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, prefetch_factor=3*batch_size, num_workers=3, pin_memory=True, persistent_workers=True, drop_last=False)
 
# test data set is built per video, so load differently
test_dataloaders = []
for video_dataset in test_dataset:
    test_dataloader = DataLoader(video_dataset, batch_size=batch_size, shuffle=False, prefetch_factor=3*batch_size, num_workers=3, pin_memory=True, persistent_workers=True, drop_last=False)
    test_dataloaders.append(test_dataloader)
print("Dataset loaded ...")

Dataset loaded ...


## Build some metrics

In [10]:
# solve class imbalance
class_weights = get_weight_balancing(case=dataset_variant)
if 'crossval' in dataset_variant:
    tool_weight   = class_weights[kfold]['tool']
    verb_weight   = class_weights[kfold]['verb']
    target_weight = class_weights[kfold]['target']
else:
    tool_weight   = class_weights['tool']
    verb_weight   = class_weights['verb']
    target_weight = class_weights['target']

# Or constant weights from average of the random sampling of the dataset: we found this to produce better result.
tool_weight     = [0.93487068, 0.94234964, 0.93487068, 1.18448115, 1.02368339, 0.97974447]
verb_weight     = [0.60002400, 0.60002400, 0.60002400, 0.61682467, 0.67082683, 0.80163207, 0.70562823, 2.11208448, 2.69230769, 0.60062402]
target_weight   = [0.49752894, 0.52041527, 0.49752894, 0.51394739, 2.71899565, 1.75577963, 0.58509403, 1.25228034, 0.49752894, 2.42993134, 0.49802647, 0.87266576, 1.36074165, 0.50150917, 0.49802647]


# model initialization
# model = network.Rendezvous('resnet18', hr_output=hr_output, use_ln=use_ln).cuda()
model = Rendezvous('resnet18', hr_output=hr_output, use_ln=use_ln).cuda()
# model = Rendezvous('efficientnet-b0', hr_output=hr_output, use_ln=use_ln).cuda() # another basemodel
pytorch_total_params = sum(p.numel() for p in model.parameters()) # total number of parameters in the model
pytorch_train_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # total number of trainable parameters in the model


# performance tracker for hyper-parameter tuning
benchmark = torch.nn.Parameter(torch.tensor([0.0]), requires_grad=False)
print("Model built ...")

# loss functions
activation  = nn.Sigmoid()

# BCEWithLogitsLoss: 
# Binary Cross-Entropy Loss with logits, which combines a sigmoid layer and the binary cross-entropy loss in one function
# used for multi-label classification tasks
loss_fn_i   = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(tool_weight).cuda())
loss_fn_v   = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(verb_weight).cuda())
loss_fn_t   = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(target_weight).cuda())

# loss function for the triplet without class weights
loss_fn_ivt = nn.BCEWithLogitsLoss()

import numpy as np
np.float = np.float64
np.int = int

##%% evaluation metrics
mAP = ivtmetrics.Recognition(100)
mAP.reset_global()
if not set_chlg_eval:
    mAPi = ivtmetrics.Recognition(6)
    mAPv = ivtmetrics.Recognition(10)
    mAPt = ivtmetrics.Recognition(15)
    mAPi.reset_global()
    mAPv.reset_global()
    mAPt.reset_global()
print("Metrics built ...")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]


Model built ...
Metrics built ...


## Optimizer and load model's weight

In [11]:
# optimizer and lr scheduler
wp_lr           = [lr/power for lr in learning_rates] # scaling the initial learning rates

module_i        = list(set(model.parameters()) - set(model.encoder.cagam.parameters()) - set(model.encoder.bottleneck.parameters()) - set(model.decoder.parameters()))
module_ivt      = list(set(model.encoder.bottleneck.parameters()).union(set(model.decoder.parameters())))
module_vt       = model.encoder.cagam.parameters()

optimizer_i     = torch.optim.SGD(module_i, lr=wp_lr[0], weight_decay=weight_decay)
scheduler_ia    = torch.optim.lr_scheduler.LinearLR(optimizer_i, start_factor=power, total_iters=warmups[0])
scheduler_ib    = torch.optim.lr_scheduler.ExponentialLR(optimizer_i, gamma=decay_rate)
scheduler_i     = torch.optim.lr_scheduler.SequentialLR(optimizer_i, schedulers=[scheduler_ia, scheduler_ib], milestones=[warmups[0]+1])

optimizer_vt    = torch.optim.SGD(module_vt, lr=wp_lr[1], weight_decay=weight_decay)
scheduler_vta   = torch.optim.lr_scheduler.LinearLR(optimizer_vt, start_factor=power, total_iters=warmups[1])
scheduler_vtb   = torch.optim.lr_scheduler.ExponentialLR(optimizer_vt, gamma=decay_rate)
scheduler_vt    = torch.optim.lr_scheduler.SequentialLR(optimizer_vt, schedulers=[scheduler_vta, scheduler_vtb], milestones=[warmups[1]+1])

optimizer_ivt   = torch.optim.SGD(module_ivt, lr=wp_lr[2], weight_decay=weight_decay)
scheduler_ivta  = torch.optim.lr_scheduler.LinearLR(optimizer_ivt, start_factor=power, total_iters=warmups[2])
scheduler_ivtb  = torch.optim.lr_scheduler.ExponentialLR(optimizer_ivt, gamma=decay_rate)
scheduler_ivt   = torch.optim.lr_scheduler.SequentialLR(optimizer_ivt, schedulers=[scheduler_ivta, scheduler_ivtb], milestones=[warmups[2]+1])

lr_schedulers   = [scheduler_i, scheduler_vt, scheduler_ivt]
optimizers      = [optimizer_i, optimizer_vt, optimizer_ivt]


# 6.27: after changing basemodel, filter the unmatched weights and load them
# prevent loading mismatched parameters and ensure only compatible weights
def load_filtered_state_dict(model, state_dict):
    model_state_dict = model.state_dict()
    filtered_state_dict = {k: v for k, v in state_dict.items() if k in model_state_dict and model_state_dict[k].shape == v.shape}
    model_state_dict.update(filtered_state_dict)
    model.load_state_dict(model_state_dict)

##%% checkpoints/weights
if os.path.exists(ckpt_path):
    # modified
    state_dict = torch.load(ckpt_path)
    load_filtered_state_dict(model, state_dict)
#     model.load_state_dict(torch.load(ckpt_path)) # original
#     model.load_state_dict(torch.load(ckpt_path), strict=False)  # modify basemodel: strict=False: ignore mismatches.

    resume_ckpt = ckpt_path # original
    
elif os.path.exists(pretrain_dir):
    pretrained_dict = torch.load(pretrain_dir)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model.state_dict().update(pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    resume_ckpt = pretrain_dir
    
# modify basemodel
# if os.path.exists(ckpt_path):
#     state_dict = torch.load(ckpt_path)
#     load_filtered_state_dict(model, state_dict)
#     resume_ckpt = ckpt_path
# else:
#     print("未找到检查点文件，使用默认的预训练 EfficientNet 权重。")

print("Model's weight loaded ...")

Model's weight loaded ...


## Training and Testing

Due to the extensive output content from the training and testing functions, only the results from 1 epoch is used below to demonstrate the training and testing process of the model.

In [12]:
# log config
# header1 = "** Run: {} | Framework: PyTorch | Method: {} | Version: {} | Data: CholecT50 | Batch: {} **".format(os.path.basename(__file__), modelname, version, batch_size)
# header2 = "** Time: {} | Start: {}-epoch  {}-steps | Init CKPT: {} | Save CKPT: {} **".format(time.ctime(), 0, 0, resume_ckpt, ckpt_path)
# header3 = "** LR Config: Init: {} | Peak: {} | Warmup Epoch: {} | Rise: {} | Decay {} | train params {} | all params {} **".format([float(f'{sch.get_last_lr()[0]:.6f}') for sch in lr_schedulers], [float(f'{v:.6f}') for v in wp_lr], warmups, power, decay_rate, pytorch_train_params, pytorch_total_params)
# maxlen  = max(len(header1), len(header2), len(header3))
# header1 = "{}{}{}".format('*'*((maxlen-len(header1))//2+1), header1, '*'*((maxlen-len(header1))//2+1) )
# header2 = "{}{}{}".format('*'*((maxlen-len(header2))//2+1), header2, '*'*((maxlen-len(header2))//2+1) )
# header3 = "{}{}{}".format('*'*((maxlen-len(header3))//2+1), header3, '*'*((maxlen-len(header3))//2+1) )
# maxlen  = max(len(header1), len(header2), len(header3))
# print("\n\n\n{}\n{}\n{}\n{}\n{}".format("*"*maxlen, header1, header2, header3, "*"*maxlen), file=open(logfile, 'a+'))
# print("Experiment started ...\n   logging outputs to: ", logfile)

print(f"is_train: {is_train}, is_test: {is_test}")
is_train = True
is_test = True

# training
if is_train:
    for epoch in range(0,epochs):
        try:
            print(f"Training | lr: {[lr.get_last_lr() for lr in lr_schedulers]} | epoch {epoch} | ")
            train_loop(train_dataloader, model, activation, loss_fn_i, loss_fn_v, loss_fn_t, loss_fn_ivt, optimizers, lr_schedulers, epoch)

            # validation
            if epoch % val_interval == 0:
                start = time.time()  
                mAP.reset_global()
#                 print("Evaluating @ epoch: ", epoch, file=open(logfile, 'a+'))
                print(f"Evaluating @ epoch: {epoch}")
#                 print("\t\t\t\t\t\t\t video-wise | eta {:.2f} secs | mAP => ivt: [{:.5f}] ".format( (time.time() - start), mAP.compute_video_AP('ivt', ignore_null=set_chlg_eval)['mAP']), file=open(logfile, 'a+'))      
#                 print(f"Video-wise | eta {(time.time() - start):.2f} secs | mAP => ivt: [{mAP.compute_video_AP('ivt', ignore_null=set_chlg_eval)['mAP']:.5f}]")
                
                # save model checkpoint
                torch.save(model.state_dict(), '/kaggle/working/model_epoch_{}.pth'.format(epoch))
        except KeyboardInterrupt:
#             print(f'>> Process cancelled by user at {time.ctime()}, ...', file=open(logfile, 'a+'))    
            sys.exit(1)
    test_ckpt = ckpt_path
    
# save the final model after training
torch.save(model.state_dict(), '/kaggle/working/model_final.pth')

# test
test_ckpt = "/kaggle/working/model_epoch_0.pth" # just for showing the output for 1 epoch
if is_test:
    total_top5_accuracy = 0.0
    total_top10_accuracy = 0.0
    total_samples = 0
#     print("Test weight used: ", test_ckpt)
    model.load_state_dict(torch.load(test_ckpt))
    
    mAP.reset_global()
    for test_dataloader in test_dataloaders:
#         test_loop(test_dataloader, model, activation, final_eval=True)
        top5_accuracy, top10_accuracy, num_samples = test_loop(test_dataloader, model, activation, final_eval=True)
        total_top5_accuracy += top5_accuracy * num_samples
        total_top10_accuracy += top10_accuracy * num_samples
        total_samples += num_samples

    total_top5_accuracy /= total_samples
    total_top10_accuracy /= total_samples
    
    if set_chlg_eval:
        mAP_i = mAP.compute_video_AP('i', ignore_null=set_chlg_eval)
        mAP_v = mAP.compute_video_AP('v', ignore_null=set_chlg_eval)
        mAP_t = mAP.compute_video_AP('t', ignore_null=set_chlg_eval)
    else:
        mAP_i = mAPi.compute_video_AP(ignore_null=set_chlg_eval)
        mAP_v = mAPv.compute_video_AP(ignore_null=set_chlg_eval)
        mAP_t = mAPt.compute_video_AP(ignore_null=set_chlg_eval)
    mAP_iv = mAP.compute_video_AP('iv', ignore_null=set_chlg_eval)
    mAP_it = mAP.compute_video_AP('it', ignore_null=set_chlg_eval)
    mAP_ivt = mAP.compute_video_AP('ivt', ignore_null=set_chlg_eval) 
    
#     print('-'*50, file=open(logfile, 'a+'))
#     print('Test Results\nPer-category AP: ', file=open(logfile, 'a+'))
#     print(f'I   : {mAP_i["AP"]}', file=open(logfile, 'a+'))
#     print(f'V   : {mAP_v["AP"]}', file=open(logfile, 'a+'))
#     print(f'T   : {mAP_t["AP"]}', file=open(logfile, 'a+'))
#     print(f'IV  : {mAP_iv["AP"]}', file=open(logfile, 'a+'))
#     print(f'IT  : {mAP_it["AP"]}', file=open(logfile, 'a+'))
#     print(f'IVT : {mAP_ivt["AP"]}', file=open(logfile, 'a+'))
#     print('-'*50, file=open(logfile, 'a+'))
#     print(f'Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT ', file=open(logfile, 'a+'))
#     print(f':::::: : {mAP_i["mAP"]:.4f} | {mAP_v["mAP"]:.4f} | {mAP_t["mAP"]:.4f} | {mAP_iv["mAP"]:.4f} | {mAP_it["mAP"]:.4f} | {mAP_ivt["mAP"]:.4f} ', file=open(logfile, 'a+'))
#     print('='*50, file=open(logfile, 'a+'))
#     print("Test results saved @ ", logfile)

    # direct print
    print('-'*50)
    print('Test Results\nPer-category AP:')
    print(f'I   : {mAP_i["AP"]}')
    print(f'V   : {mAP_v["AP"]}')
    print(f'T   : {mAP_t["AP"]}')
    print(f'IV  : {mAP_iv["AP"]}')
    print(f'IT  : {mAP_it["AP"]}')
    print(f'IVT : {mAP_ivt["AP"]}')
    print('-'*50)
    print(f'Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT ')
    print(f':::::: : {mAP_i["mAP"]:.4f} | {mAP_v["mAP"]:.4f} | {mAP_t["mAP"]:.4f} | {mAP_iv["mAP"]:.4f} | {mAP_it["mAP"]:.4f} | {mAP_ivt["mAP"]:.4f}')
    print('='*50)

    print(f"Top-5 Accuracy: {total_top5_accuracy:.4f}")
    print(f"Top-10 Accuracy: {total_top10_accuracy:.4f}")

# print("All done!\nShutting done...\nIt is what it is ...\nC'est finis! {}".format("-"*maxlen) , file=open(logfile, 'a+'))
# print("All done!\nShutting done...\nIt is what it is ...\nC'est finis! {}".format("-"*maxlen))

is_train: False, is_test: False
Training | lr: [[0.01], [0.01], [0.01]] | epoch 0 | 
Batch 1/1970 | Losses => i: [0.8923] v: [0.8756] t: [0.9071] ivt: [0.9911]
Batch 2/1970 | Losses => i: [1.2241] v: [0.8715] t: [0.9329] ivt: [0.9731]
Batch 3/1970 | Losses => i: [1.3250] v: [0.8860] t: [0.9399] ivt: [0.9397]
Batch 4/1970 | Losses => i: [1.3352] v: [0.8727] t: [0.9348] ivt: [0.9154]
Batch 5/1970 | Losses => i: [1.2936] v: [0.8617] t: [0.9531] ivt: [0.8872]
Batch 6/1970 | Losses => i: [1.4102] v: [0.8699] t: [0.9457] ivt: [0.8835]
Batch 7/1970 | Losses => i: [1.3589] v: [0.8744] t: [0.9568] ivt: [0.8450]
Batch 8/1970 | Losses => i: [1.3948] v: [0.8647] t: [0.9543] ivt: [0.8268]
Batch 9/1970 | Losses => i: [1.3099] v: [0.8654] t: [0.9586] ivt: [0.8053]
Batch 10/1970 | Losses => i: [1.2676] v: [0.8586] t: [0.9465] ivt: [0.7818]
Batch 11/1970 | Losses => i: [1.2741] v: [0.8509] t: [0.9353] ivt: [0.7539]
Batch 12/1970 | Losses => i: [1.2427] v: [0.8525] t: [0.9351] ivt: [0.7338]
Batch 13/197

The following are the results collected from various ablation experiments.

In [13]:
# Results of ablation study on the Improved WSL Module

# # Original WSL:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8534 | 0.5683 | 0.3776 | 0.2993 | 0.2716 | 0.2047
# # WSL + Conv2:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8611 | 0.5676 | 0.3782 | 0.2989 | 0.2718 | 0.2094
# # WSL + Attention:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8867 | 0.5681 | 0.3780 | 0.3028 | 0.2802 | 0.2061
# # WSL + Conv2 + Attention:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8952 | 0.5685 | 0.3779 | 0.3031 | 0.2794 | 0.2108
                            
                
        
# Results of ablation study on the Improved CAGAM Module

# # Original CAGAM:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8527 | 0.5681 | 0.3774 | 0.2992 | 0.2713 | 0.2038
# # CAGAM + Muti-Head Attention:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8533 | 0.5883 | 0.3927 | 0.3002 | 0.2778 | 0.2141
# # CAGAM + Batch Normalization
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8531 | 0.5726 | 0.3825 | 0.2984 | 0.2723 | 0.2071
# # CAGAM + Both Improvement
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8529 | 0.5923 | 0.3961 | 0.3011 | 0.2804 | 0.2133
                            

    
# Results of ablation study on the Improved MHMA Module

# # Original MHMA:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8525 | 0.5687 | 0.3773 | 0.2986 | 0.2726 | 0.2043
# # MHMA + LSTM:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8602 | 0.5671 | 0.3788 | 0.3144 | 0.2965 | 0.2261
                            

    
# # Results of ablation study on the Dynamic Convolution (DC) Module

# # Rendezvous:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8531 | 0.5683 | 0.3784 | 0.2993 | 0.2718 | 0.2052
# ==================================================
# Top-5 Accuracy: 0.7633
# Top-10 Accuracy: 0.8871
# ==================================================

# # Rendezvous + DC:
# --------------------------------------------------
# Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT 
# :::::: : 0.8544 | 0.5710 | 0.3792 | 0.3076 | 0.2853 | 0.2382
# ==================================================
# Top-5 Accuracy: 0.7814
# Top-10 Accuracy: 0.9019
# ==================================================